# Knapsack Problem


En este notebook vamos a resolver el problema del "Knapsack Problem" utilizando un algoritmo de tensor networks.

En este problema se tiene un conjunto de objetos con un peso y un valor, y una mochila con una capacidad limitada. El objetivo es seleccionar los objetos que se pueden meter en la mochila de manera que se maximice el valor total, sin superar la capacidad de la mochila.


In [1]:
import numpy as np

from IPython.display import HTML, display



* Función global
* Función que genere los tensores
    * Función que determine qué tensores hay que crear
    * Función que cree cada tensor
* Función que contraiga los tensores


---
# Función que genera los tensores


In [2]:
def tensor_initial_generator(peso:int, n_elementos:int, capacidad:int, weight_partial:int, tau:float=0)->np.array:
    """
    Funcion que genera el primer tensor de la cadena.
    Parameters
    ----------
    peso : int
        Peso del objeto.
    n_elementos : int
        Número de objetos disponibles.
    capacidad : int
        Capacidad de la mochila.
    position : int
        Posición del tensor en la lista.
    weight_partial : int
        Peso parcial de la mochila hasta este punto.

    Returns
    -------
    np.array
        Tensor inicial.
    """

    n_possible = min((capacidad - weight_partial)//peso, n_elementos) + 1
    max_weight_partial = min(capacidad + 1, (n_possible-1)*peso + weight_partial+1)

    # Inicializamos el tensor
    tensor = np.zeros((n_possible, max_weight_partial), dtype=float)

    # Rellenamos el tensor
    for n in range(n_possible):
        weight = weight_partial + peso*n
        tensor[n,weight] = 1#np.exp(tau*n/n_elementos)

    return tensor

In [3]:
def tensor_intermediate_generator(peso:int, n_elementos:int, capacidad:int, max_weight_partial:int, tau:float=0)->np.array:
    """
    Funcion que genera el tensor intermedio de la cadena.
    Parameters
    ----------
    peso : int
        Peso del objeto.
    n_elementos : int
        Número de objetos disponibles.
    capacidad : int
        Capacidad de la mochila.

    Returns
    -------
    np.array
        Tensor intermedio.
    """
    # Número de posibles elementos
    max_weight = min(capacidad+1, n_elementos*peso + max_weight_partial)

    # Inicializamos el tensor
    tensor = np.zeros((max_weight_partial, max_weight), dtype=float)

    # Rellenamos el tensor
    for weight_partial in range(max_weight_partial):
        n_possible = min((capacidad - weight_partial)//peso, n_elementos) + 1
        for n in range(n_possible):
            weight_down = weight_partial + peso*n
            tensor[weight_partial,weight_down] = 1#np.exp(tau*n/n_elementos)

    return tensor

In [4]:

def tensor_final_generator(peso:int, n_elementos:int, capacidad:int, max_weight_partial:int, tau:float, extreme:bool=False)->np.array:
    """
    Funcion que genera el tensor final de la cadena.
    Parameters
    ----------
    peso : int
        Peso del objeto.
    n_elementos : int
        Número de objetos disponibles.
    capacidad : int
        Capacidad de la mochila.
    max_weight_partial : int
        Peso parcial máximo de la mochila hasta este punto.
    extreme:
        Si es True, solo permite el valor exacto final.

    Returns
    -------
    np.array
        Tensor final.
    """
    # Inicializamos el tensor
    tensor = np.zeros((max_weight_partial), dtype=float)

    # Rellenamos el tensor
    for weight_partial in range(max_weight_partial):
        n_possible = min((capacidad - weight_partial)//peso, n_elementos) + 1
        if extreme:
            weight_down = weight_partial + peso*(n_possible-1)
            if weight_down == capacidad:
                tensor[weight_partial] += 1
        else:
            for n in range(n_possible):
                weight_down = weight_partial + peso*n
                if weight_partial == max_weight_partial-1 and n == n_possible-1:
                    tensor[weight_partial] += np.exp(-tau*(capacidad - weight_down))*1#np.exp(tau*n/n_elementos)
                else:
                    tensor[weight_partial] += 0#np.exp(tau*n/n_elementos)

    return tensor



Función que genera un tensor genérico.

In [5]:
def tensor_generator(pesos:np.array, n_elementos:np.array, capacidad:int, tau:float, extreme:bool=False)->list:
    """
    Funcion que genera los tensores para la contraccion.
    Parameters
    ----------
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.
    n_elementos : np.array
        Array del número de objetos disponibles para cada clase.
    capacidad : int
        Capacidad de la mochila.
    extreme:
        Si es True, solo permite el valor exacto final.

    Returns
    -------
    list
        Lista de los tensores a utilizar.
    """
    # Número de clases de objetos
    n_clases = len(pesos)

    # Lista de los tensores
    lista_tensores = []
    append = lista_tensores.append

    # Primer tensor
    tensor = tensor_initial_generator(pesos[0], n_elementos[0], capacidad, 0, tau=tau)
    append(tensor)

    # Tensores intermedios
    for position in range(1, n_clases-1):
        tensor = tensor_intermediate_generator(pesos[position], n_elementos[position], capacidad, tensor.shape[1], tau=tau)
        append(tensor)

    # Último tensor
    append(tensor_final_generator(pesos[-1], n_elementos[-1], capacidad, tensor.shape[1], tau, extreme=extreme))

    return lista_tensores

---
# Función de contracción

In [6]:
def contraction_tensors(lista_tensores:list):

    # Numero de tensores
    n_tensors = len(lista_tensores)
    vector = lista_tensores[-1]
    tensores_intermedios = [vector]
    append = tensores_intermedios.append

    for i in range(n_tensors-2, -1, -1):
        vector = lista_tensores[i] @ vector
        append(vector)

    tensores_intermedios.reverse()
    return vector, tensores_intermedios
    


---
# Funcion de peso parcial

In [7]:
def weight_solution(solution:np.array, pesos:np.array)->int:
    """
    Funcion que calcula el peso parcial de la mochila.
    Parameters
    ----------
    solution : np.array
        Solucion del problema.
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.

    Returns
    -------
    int
        Peso parcial de la mochila.
    """
    return np.dot(solution, pesos)



---
# Función global
	

Esta función es la que se encarga de resolver el problema del Knapsack utilizando un algoritmo de tensor networks.


In [8]:
def knapsack_solver(pesos:np.array, n_elementos:np.array, capacidad:int, tau:float, extreme:bool=False)->np.array:
    """
    Funcion que resuelve el problema del Knapsack utilizando un algoritmo de tensor networks.
    Parameters
    ----------
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.
    n_elementos : np.array
        Array del número de objetos disponibles para cada clase.
    capacidad : int
        Capacidad de la mochila.
    extreme:
        Si es True, solo permite el valor exacto final.

    Returns
    -------
    np.array
        Número de elementos escogidos para cada clase de los objetos seleccionados.
    """
    # Número de clases de objetos
    n_clases = len(pesos)

    # Array de la solucion
    solution = np.ones(n_clases, dtype=int)*(-1)

    if np.dot(n_elementos, pesos) <= capacidad:
        solution = np.array(n_elementos, dtype=int)
        return solution


    # Lista de los tensores a utilizar
    lista_tensores = tensor_generator(pesos, n_elementos, capacidad, tau, extreme=extreme)
    for t in lista_tensores:
        print(t.shape)
        print(t)



    # Paso inicial de la iteración
    vector_de_salida, tensores_intermedios = contraction_tensors(lista_tensores)



    # Obtenemos el valor correcto para esta variable
    solution[0] = np.argmax(vector_de_salida)

    # Proceso iterativo para el resto de variables
    for position in range(1, n_clases-1):
        # Peso de la solucion hasta este paso
        weight_partial = weight_solution(solution[:position], pesos[:position])
     
        if weight_partial == capacidad: break

        # Creamos el primer tensor de la cadena
        tensor_inicial = tensor_initial_generator(pesos[position], n_elementos[position], capacidad, weight_partial, tau=tau)
        # Contraccion aprovechando los tensores intermedios
       
        vector_de_salida = tensor_inicial @ tensores_intermedios[position+1][:tensor_inicial.shape[1]]

        # Obtenemos el valor correcto para esta variable
        solution[position] = np.argmax(vector_de_salida)


    # Obtenemos el valor de la última variable
    weight_partial = weight_solution(solution[:-1], pesos[:-1])
    solution[-1] = max( min((capacidad - weight_partial)//pesos[-1], n_elementos[-1]), 0)

    return solution

---
# Prueba

In [9]:
n_clases = 1000
capacidad = 987
np.random.seed(1)
pesos = np.random.randint(1,10, n_clases)
print(pesos)
#pesos = np.ones(n_clases, dtype=int)
n_elementos = np.array([1,]*n_clases) #np.random.randint(1,5, n_clases)

tau = 2
solution = knapsack_solver(pesos, n_elementos, capacidad, tau, extreme=True)

print('La solucion es: ', solution)
print('El peso de la solucion es: ', weight_solution(solution, pesos))

[6 9 6 1 1 2 8 7 3 5 6 3 5 3 5 8 8 2 8 1 7 8 7 2 1 2 9 9 4 9 8 4 7 6 2 4 5
 9 2 5 1 4 3 1 5 3 8 8 9 7 4 8 8 5 6 4 7 9 1 3 8 8 8 4 1 9 8 8 2 2 4 1 9 7
 5 6 7 3 6 8 9 5 5 8 8 5 1 3 1 8 2 8 9 5 1 2 9 3 4 2 3 8 3 7 1 3 7 7 3 8 8
 1 7 6 2 5 7 1 7 6 2 3 2 6 5 1 8 9 6 8 1 4 2 5 5 7 9 9 3 8 6 6 5 6 9 6 9 2
 2 9 8 1 4 5 3 1 4 6 2 3 5 4 1 7 1 8 3 9 4 1 9 5 3 1 4 9 2 5 4 4 7 8 4 6 4
 3 5 5 1 4 4 9 4 6 7 8 6 2 8 1 3 9 3 2 5 1 5 2 8 4 2 7 7 7 7 1 1 3 7 1 7 8
 1 4 1 4 5 8 6 4 9 9 1 7 8 6 5 6 3 6 7 7 9 8 8 8 3 7 1 6 3 2 9 6 5 2 3 1 5
 8 1 7 3 5 4 7 8 7 4 1 7 5 8 7 3 6 9 7 5 3 5 1 1 4 5 4 2 3 6 5 1 9 3 4 5 5
 9 3 2 7 4 9 8 1 6 3 3 9 6 1 6 9 7 7 1 5 8 4 1 2 7 1 7 2 7 5 3 6 5 7 3 3 8
 6 1 8 9 9 9 1 8 3 1 8 2 2 6 2 6 7 5 9 8 6 2 9 1 6 4 1 5 9 7 3 5 4 3 1 1 5
 3 6 1 1 4 9 6 4 2 5 8 4 3 3 3 7 7 1 2 6 7 6 9 9 6 6 8 6 2 4 4 4 4 7 2 4 1
 6 1 6 3 8 7 5 1 3 5 9 8 7 8 8 2 8 8 4 9 4 1 7 4 1 7 6 7 5 7 7 3 3 5 2 3 4
 4 7 8 1 4 4 7 9 7 6 2 4 3 7 4 7 8 3 9 1 2 9 7 1 1 2 3 8 8 5 5 1 2 1 9 6 7
 3 6 5 4 1 7 3 2 5 5 1 9 

# Comparativa

In [10]:
from time import time
time_total=time()
Q=100
elementos = 100
encontrado=False
cantidad = 12
np.random.seed(1)
#pesos = np.random.randint(1, 11, size=elementos)
print(len(pesos))
pesos.sort()
print(pesos)
Co= np.zeros((cantidad+1, 2))

for j in range(0,elementos):
    inicio=time()
    for i in range(0,Co.shape[0]):
        Co[i][0]=pesos[j]*(i%(cantidad+1))+Co[i][0]
        
        if Co[i][0]==Q:
            Co[i][j+1]=(i%(cantidad+1))
            L=Co[i]
            Co=0
            print(f"La fila con el valor más alto en la columna {0} es: {L}")
            encontrado=True
            break

        
        if Co[i][0]<Q:
            Co[i][j+1]=(i%(cantidad+1))
        else:
            Co[i][0]=-1
       
        if Co[i][0] != 0:
            for n in range(Co.shape[0]):
                if n != i and Co[i][0] == Co[n][0] and i>n:
                    Co[i][0]=-1

    if encontrado==True:
        break  
    Co = Co[Co[:, 0] != -1]
    
    print("cambio de objeto",j)
    fin=time()
    print(f"Tiempo de ejecución: {fin-inicio} segundos")
    #for i in range(0,Co.shape[0]):
        #print(np.round(Co[i]).astype(int))
    print(Co.shape[0])
    Co = np.repeat(Co, cantidad+1, axis=0)
    column_of_zeros = np.zeros((Co.shape[0], 1))  # Columna de ceros
    Co = np.hstack((Co, column_of_zeros))
    
indice_fila_max = np.argmax(Co[:, 0])

# Paso 2: Seleccionar la fila correspondiente al índice encontrado
fila_maxima = Co[indice_fila_max]

print(f"La fila con el valor más alto en la columna {0} es: {fila_maxima}")
fin_t=time()
print(f"Tiempo total de ejecución: {fin_t-time_total} segundos")

1000
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5

TypeError: 'int' object is not subscriptable